# 数据读取

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime 
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from tqdm import tqdm
from numpy import concatenate
from pandas import DataFrame
from pandas import concat
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from __future__ import division  
from sklearn.metrics import mean_absolute_error,mean_squared_error
import requests
import matplotlib.pyplot as plt

/home/liyouru/virtual_tf1.3_python2.7/local/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
base_path = '/home/liyouru/.ipython/profile_myserver/dataset/KDD_Cup2018/'
p_bj_aq = base_path + 'bj_aq_split/' + 'bj_17_18_aq_split_'
p_bj_meo = base_path + 'bj_meo_gen/' + 'beijing_17_18_meo_gen_'
n_p_bj_meo = base_path + 'bj_meo_gen/' + 'n_beijing_17_18_meo_gen_'
p_ld_aq = base_path + 'ld_aq_split/' + 'ld_17_18_aq_split_'
p_ld_meo = base_path + 'ld_meo_gen/' + 'london_17_18_meo_gen_'
n_p_ld_meo = base_path + 'ld_meo_gen/' + 'n_london_17_18_meo_gen_'
p_index = base_path + 'station_list.csv'

df_index = pd.read_csv(p_index)
bj_index = list(df_index['beijing'])
ld_index = list(df_index['london'])[:13]

# 基本特征提取合并

* 北京

In [3]:
def get_intersection(a,b):
    return list(set(a).intersection(set(b)))  
    
def get_aq_feature(df,time_list):
    t_feature = [[],[],[],[],[],[]]
    for each in df.iterrows():
        if each[1]['utc_time'] in time_list:
            t_feature[0].append(each[1]['PM2.5'])
            t_feature[1].append(each[1]['PM10'])
            t_feature[2].append(each[1]['NO2'])
            t_feature[3].append(each[1]['CO'])
            t_feature[4].append(each[1]['O3'])
            t_feature[5].append(each[1]['SO2'])
    return t_feature

def get_meo_feature(df,time_list):
    t_feature = [[],[],[],[],[]]
    for each in df.iterrows():
        if each[1]['utc_time'] in time_list:
            t_feature[0].append(each[1]['temperature'])
            t_feature[1].append(each[1]['pressure'])
            t_feature[2].append(each[1]['humidity'])
            t_feature[3].append(each[1]['wind_direction'])
            t_feature[4].append(each[1]['wind_speed/kph'])
    return t_feature

def get_time_feature(df,time_list):
    t_feature = [[],[]]
    for each in df.iterrows():
        if each[1]['utc_time'] in time_list:
            # 周末时段
            t_feature[0].append(datetime.strptime(each[1]['utc_time'][:10], '%Y-%m-%d').weekday())
            # 高峰时段
            if each[1]['utc_time'][11:13] in ['00','01','02','09','10','11']:
                t_feature[1].append(1)
            else:
                t_feature[1].append(0)
    return t_feature

In [4]:
for each in tqdm(bj_index):
    t_feature = []
    # load file
    t_df_aq = pd.read_csv(p_bj_aq + each + '.csv')
    del t_df_aq['Unnamed: 0']
    t_df_aq = t_df_aq.drop_duplicates(['utc_time'])
    t_df_meo = pd.read_csv(p_bj_meo + each + '.csv')
    t_df_meo = t_df_meo.append(pd.read_csv(n_p_bj_meo + each + '.csv'))
    del t_df_meo['Unnamed: 0']
    t_df_meo = t_df_meo.drop_duplicates(['utc_time'])
    # get time intersection index
    t_time_index = get_intersection(list(t_df_aq['utc_time']),list(t_df_meo['utc_time']))
    # get value and feature
    t_feature_aq = get_aq_feature(t_df_aq,t_time_index)
    t_feature_meo = get_meo_feature(t_df_meo,t_time_index)
    t_feature_time = get_time_feature(t_df_meo,t_time_index)
    t_feature = t_feature_aq
    t_feature.extend(t_feature_meo)
    t_feature.extend(t_feature_time)
    # feature to csv
    t_col_index = ['PM2.5', 'PM10', 'NO2', 'CO', 'O3', 'SO2','temperature', 'pressure', 'humidity','wind_direction', 'wind_speed/kph','weekday','rush_time']
    t_gen_df = DataFrame(np.transpose(t_feature) ,index = None, columns = t_col_index)
    t_gen_df.to_csv(base_path + 'bj_feature_s1/'+ each +'_base.csv')

100%|██████████| 35/35 [04:38<00:00,  7.96s/it]


* 伦敦

In [5]:
def get_aq_feature_ld(df,time_list):
    t_feature = [[],[],[]]
    for each in df.iterrows():
        if time_format(each[1]['MeasurementDateGMT']) in time_list:
            t_feature[0].append(each[1]['PM2.5 (ug/m3)'])
            t_feature[1].append(each[1]['PM10 (ug/m3)'])
            t_feature[2].append(each[1]['NO2 (ug/m3)'])
    return t_feature

def get_meo_feature_ld(df,time_list):
    t_feature = [[],[],[],[],[]]
    for each in df.iterrows():
        if each[1]['utc_time'] in time_list:
            t_feature[0].append(each[1]['temperature'])
            t_feature[1].append(each[1]['pressure'])
            t_feature[2].append(each[1]['humidity'])
            t_feature[3].append(each[1]['wind_direction'])
            t_feature[4].append(each[1]['wind_speed/kph'])
    return t_feature

def get_time_feature_ld(df,time_list):
    t_feature = [[],[]]
    for each in df.iterrows():
        time = time_format(each[1]['utc_time'])
        if time in time_list:
            # 周末时段
            t_feature[0].append(datetime.strptime(time[:10], '%Y-%m-%d').weekday())
            # 高峰时段
            if time[11:13] in ['08','09','10','17','18','19']:
                t_feature[1].append(1)
            else:
                t_feature[1].append(0)
    return t_feature

def time_format(t_str):
    if len(t_str) != 19:
        time = datetime.strptime(t_str, '%Y/%m/%d %H:%M')
        n_t = time.strftime('%Y-%m-%d %H:%M:%S')
    else:
        n_t = t_str
    return n_t

def time_list_format_trans(t_list):
    time = []
    for each in t_list:
        time.append(time_format(each))
    return time

In [6]:
for each in tqdm(ld_index):
    t_feature = []
    # load file
    t_df_aq = pd.read_csv(p_ld_aq + each + '.csv')
    del t_df_aq['Unnamed: 0']
    t_df_aq = t_df_aq.drop_duplicates(['MeasurementDateGMT'])
    t_df_meo = pd.read_csv(p_ld_meo + each + '.csv')
    t_df_meo = t_df_meo.append(pd.read_csv(n_p_ld_meo + each + '.csv'))
    del t_df_meo['Unnamed: 0']
    t_df_meo = t_df_meo.drop_duplicates(['utc_time'])
    # get time intersection index
    t_time_index = get_intersection(time_list_format_trans(list(t_df_aq['MeasurementDateGMT'])),list(t_df_meo['utc_time']))
    # get value and feature
    t_feature_aq = get_aq_feature_ld(t_df_aq,t_time_index)
    t_feature_meo = get_meo_feature_ld(t_df_meo,t_time_index)
    t_feature_time = get_time_feature_ld(t_df_meo,t_time_index)
    t_feature = t_feature_aq
    t_feature.extend(t_feature_meo)
    t_feature.extend(t_feature_time)
    # feature to csv
    t_col_index = ['PM2.5', 'PM10', 'NO2','temperature', 'pressure', 'humidity', 'wind_direction','wind_speed/kph','weekday','rush_time']
    t_gen_df = DataFrame(np.transpose(t_feature) ,index = None, columns = t_col_index)
    t_gen_df.to_csv(base_path + 'ld_feature_s1/'+ each +'_base.csv')

100%|██████████| 13/13 [01:47<00:00,  8.29s/it]


# 滑动窗口特征提取

In [7]:
def get_bin_feature(agg,city,n_bin,n_in):
# 抽取历史窗口空气质量统计特征
    if city == 0:
        n_aq = 3
    else:
        n_aq = 6
        
    sta_feature = []
    for i in range(n_aq * 7):
        sta_feature.append([])
    # aq in df to list
    df_aq = []
    for i in range(n_aq):
        for j in range(n_in-n_bin,n_in):
            df_aq.append(list(agg['var'+str(i + 1)+'(t-'+str(j + 1)+')']))
    # get statistics feature for each aq in historcal windows
    aq = []
    for i in range(n_aq * 7):
        aq.append([])
        # sample dimention
    for i in tqdm(range(agg.shape[0])):
        aq = []
            # aq type numbers
        for j in range(n_aq):
            aq.append([])
            for k in range(j * n_bin,(j + 1) * n_bin):
                if np.isnan(df_aq[k][i]):
                    aq[j].append(0)
                else:
                    aq[j].append(df_aq[k][i])
            sta_feature[7 * j].append(statistics.mean(aq[j]))
            sta_feature[7 * j + 1].append(statistics.median(aq[j]))
            sta_feature[7 * j + 2].append(statistics.variance(aq[j]))
            sta_feature[7 * j + 3].append(statistics.stdev(aq[j]))
            sta_feature[7 * j + 4].append(max(aq[j]))
            sta_feature[7 * j + 5].append(min(aq[j]))
            sta_feature[7 * j + 6].append(max(aq[j]) - min(aq[j]))

    insert_index = 0
    for i in range(len(sta_feature)):
        agg.insert(insert_index + i,'bin'+str(n_bin)+'-'+ str(i),sta_feature[i])
    return agg

In [8]:
import statistics
    
# 将序列转化成监督学习问题
# 参数：city = 1/2 aq = 1/2/3
def series_to_supervised(data, meo, n_in = 1, n_out = 1, dropnan = True, city = 1):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    
    # 输入序列(t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    
    # 预测序列 (t, t+1, ... t+n)
    for i in range(n_out - 1, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t + %d)' % (j + 1, i)) for j in range(n_vars)]
    
    # concat输出与预测序列
    agg = concat(cols, axis = 1)
    agg.columns = names
    
    # 清除value窗口除目标变量外字段
    drop_index = []
    if city == 1:
        index = [3,4,6,12,13]
        for each in index:
            drop_index.append('var' + str(each) + '(t + '+ str(n_out - 1)+')')
    else:
        index = [3,9,10]
        for each in index:
            drop_index.append('var' + str(each) + '(t + '+ str(n_out - 1)+')')
    agg.drop(drop_index, axis = 1, inplace = True)
    
    # 清除历史窗口的气象特征
    if city == 0:
        for i in range(2,n_in + 1):
            t1 = i
            for j in range(4,8):
                t2 = j
                agg.drop('var'+str(t2)+'(t-'+str(t1)+')', axis = 1, inplace = True)
    else:
        for i in range(2,n_in + 1):
            t1 = i
            for j in range(7,11):
                t2 = j
                agg.drop('var'+str(t2)+'(t-'+str(t1)+')', axis = 1, inplace = True)
    
    # 加入对历史窗口的统计特征
    bin_index = [24,48]
    for each in bin_index:
        agg = get_bin_feature(agg,city,each,n_in)
        
    # 得到测试集特征
    test_df = agg[-48:]
    if city == 1:
        test_df.drop('var1' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var2' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var5' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var7' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var8' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var9' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var10' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var11' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
    else:
        test_df.drop('var1' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var2' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var4' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var5' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var6' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var7' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
        test_df.drop('var8' + '(t + '+ str(n_out - 1)+')',axis = 1, inplace = True)
    
    # 链接天气预报
    if city == 1:
        meo.columns = ['var7(t + 71)','var8(t + 71)','var9(t + 71)','var10(t + 71)','var11(t + 71)']
    else:
        meo.columns = ['var4(t + 71)','var5(t + 71)','var6(t + 71)','var7(t + 71)','var8(t + 71)']
    test_df = DataFrame(np.concatenate((test_df.values, meo.values), axis = 1))
    
    if dropnan:
        agg.dropna(inplace = True)
    
    # 调整value在df中的顺序
    if city == 1:
        bj_var1 = agg['var5(t + 71)']
        agg.drop(labels=['var5(t + 71)'], axis=1,inplace = True)
        agg.insert(0, 'var5(t + 71)', bj_var1)
        bj_var2 = agg['var2(t + 71)']
        agg.drop(labels=['var2(t + 71)'], axis=1,inplace = True)
        agg.insert(0, 'var2(t + 71)', bj_var2)
        bj_var5 = agg['var1(t + 71)']
        agg.drop(labels=['var1(t + 71)'], axis=1,inplace = True)
        agg.insert(0, 'var1(t + 71)', bj_var5)
    else:
        ld_var1 = agg['var2(t + 71)']
        agg.drop(labels=['var2(t + 71)'], axis=1,inplace = True)
        agg.insert(0, 'var2(t + 71)', ld_var1)
        ld_var2 = agg['var1(t + 71)']
        agg.drop(labels=['var1(t + 71)'], axis=1,inplace = True)
        agg.insert(0, 'var1(t + 71)', ld_var2)
    
    if city == 1:
        test_df.columns = list(agg.columns)[3:]
    else:
        test_df.columns = list(agg.columns)[2:]
    res = [agg,test_df]
    
    return res

# 模型定义与误差评测

In [9]:
def train_and_predict(X,y,X_t,t_model):
    # load data
    train_data = X
    train_label = y
    print "TRAINING SIZE:",train_data.shape,',',train_label.shape[0]
    
    if t_model == 'XGB': 
        # define model
        xg_train = xgb.DMatrix(train_data, label = train_label)
        # param define
        param={
                'booster':'gbtree',
                'objective': 'reg:linear', 
                'gamma':0,  # 用于控制是否后剪枝的参数,越大越保守，一般0.1、0.2这样子。
                'max_depth':4, # 构建树的深度，越大越容易过拟合
                'lambda':1,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
                'subsample':1, # 随机采样训练样本
                'colsample_bytree':0.5, # 生成树时进行的列采样
                'min_child_weight':1, 
                # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
                #，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
                #这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。 
                'silent':0 ,#设置成1则没有运行信息输出，最好是设置为0.
                'eta': 0.1, # 如同学习率
                'seed':1500,
                'nthread':8,# cpu 线程数
                'eval_metric': 'rmse'
                }
        param['eval_metric'] = ['rmse', 'map']
        watchlist = [ (xg_train,'train') ]
        #train_model
        num_round = 1000
        print 'XGBoost Model is Training...'
        bst = xgb.train(param, xg_train, num_round, watchlist,verbose_eval=500)
        # make prediction
        xg_test = xgb.DMatrix(X_t)
        result = bst.predict(xg_test)
    elif t_model == 'GBRT':
        # define model
        GBRT = GradientBoostingRegressor(loss = 'lad', n_estimators = 200, max_depth = 4,verbose=0)
        # data split
        X_train, X_test, Y_train = X,X_t,y
        # make prediction
        print 'GBRT Model is Training...'
        GBRT.fit(X_train,Y_train)
        result = GBRT.predict(X_test)
    elif t_model == 'RF':
        # define model
        RF = RandomForestRegressor(n_estimators = 200, max_depth = 4, n_jobs = -1 )
        # data split
        X_train, X_test, Y_train = X,X_t,y
        # make prediction
        print 'RF Model is Training...'
        RF.fit(X_train,Y_train)
        result = RF.predict(X_test)
    else:
        lgb_train = lgb.Dataset(X[:-48], y[:-48])
        lgb_eval = lgb.Dataset(X[-48:], y[-48:], reference=lgb_train)
        params = {
                    'task': 'train',
                    'boosting_type': 'gbdt',
                    'objective': 'binary',
                    'metric': {'l2', 'mae'},
                    'num_leaves': 31,
                    'learning_rate': 0.05,
                    'feature_fraction': 0.9,
                    'bagging_fraction': 0.8,
                    'bagging_freq': 5,
                    'verbose': 1
                }
        # train
        gbm = lgb.train(params,lgb_train,num_boost_round=20,valid_sets=lgb_eval,early_stopping_rounds=5)
        result = gbm.predict(X_t, num_iteration=gbm.best_iteration)
    return list(result)

def smape(actual, predicted):
    a = np.abs(np.array(actual) - np.array(predicted))
    b = np.array(actual) + np.array(predicted)
    return 2 * np.mean(np.divide(a, b, out=np.zeros_like(a), where=b!=0, casting='unsafe'))

# 站点对应空气污染物浓度预测

In [10]:
import time
def cal_pred_each_pos(file_list,t_model,is_test,n_pred):
    pm25,pm10,o3,aq = [],[],[],[]
    index = [pm25,pm10,o3]
    names = ['PM2.5', 'PM10', 'O3']
    if len(file_list) == 35:
        for each in file_list:
            # load file
            t_df = pd.read_csv(base_path + 'bj_feature_s1/'+ each +'_base.csv')
            del t_df['Unnamed: 0']
            # get slide feature and value
            meo_api = pd.read_csv(base_path + 'daily_data/bj_weather/'+ each +'.csv')
            api_df = meo_api[['temperature', 'pressure', 'humidity', 'wind_direction','wind_speed/kph']]
            t_res = series_to_supervised(t_df.values,api_df,48,n_pred,True,1)
            t_slide_df = t_res[0]
            t_slide_df_test = t_res[1]
            # split feature and value
            t_index = list(t_slide_df.columns)
            t_feature_index,t_value_index = t_index[3:],t_index[0:3]
            # make prediction for each value
            for i in range(3):
                if is_test:
                        t_X ,t_y = t_slide_df[t_feature_index][:-48],t_slide_df[t_value_index[i]][:-48]
                        t_X_t,t_y_t = t_slide_df[t_feature_index][-48:],t_slide_df[t_value_index[i]][-48:]
                        t0 = time.time()
                        t_value = train_and_predict(t_X,t_y,t_X_t,t_model)
                        print time.time() - t0
                        for t in range(len(t_value)): t_value[t] = int(t_value[t])
                        print 'STATION: ',each
                        print 'AIR AQ TYPE:',names[i]
                        plt.plot(list(t_y_t),'r')
                        plt.plot(t_value,'b')
                        plt.show()
                        print 'SMAPE:',smape(t_y_t,t_value)
                        print '------------------------------------'    
                else:
                    t_X ,t_y = t_slide_df[t_feature_index],t_slide_df[t_value_index[i]]
                    t_X_t = t_slide_df_test[t_feature_index]
                    t_value = train_and_predict(t_X,t_y,t_X_t,t_model)
                    for t in range(len(t_value)): t_value[t] = int(t_value[t])
                    index[i].append(t_value)
                    print 'STATION: ',each
                    print 'AIR AQ TYPE:',names[i]
                    print 'VALUE:',t_value
                    print '------------------------------------'  
    else:
        for each in file_list:
            # load file
            t_df = pd.read_csv(base_path + 'ld_feature_s1/'+ each +'_base.csv')
            del t_df['Unnamed: 0']
            # get slide feature and value
            meo_api = pd.read_csv(base_path + 'daily_data/ld_weather/'+ each +'.csv')
            api_df = meo_api[['temperature', 'pressure', 'humidity', 'wind_direction','wind_speed/kph']]
            t_res = series_to_supervised(t_df.values,api_df,48,n_pred,True,0)
            t_slide_df = t_res[0]
            t_slide_df_test = t_res[1]
            # split feature and value
            t_index = list(t_slide_df.columns)
            t_feature_index,t_value_index = t_index[2:],t_index[0:2]
            # make prediction for each value
            for i in range(3):
                if i == 2:
                    t_aq = []
                    for j in range(48):
                        t_aq.append(0)
                    index[i].append(t_aq)
                else:
                    if is_test:
                        t_X ,t_y = t_slide_df[t_feature_index][:-48],t_slide_df[t_value_index[i]][:-48]
                        t_X_t,t_y_t = t_slide_df[t_feature_index][-48:],t_slide_df[t_value_index[i]][-48:]
                        t_value = train_and_predict(t_X,t_y,t_X_t,t_model)
                        for t in range(len(t_value)): t_value[t] = int(t_value[t])
                        print 'STATION: ',each
                        print 'AIR AQ TYPE:',names[i]
                        plt.plot(list(t_y_t),'r')
                        plt.plot(t_value,'b')
                        plt.show()
                        print 'SMAPE:',smape(t_y_t,t_value)
                        print '------------------------------------'    
                    else:
                        t_X ,t_y = t_slide_df[t_feature_index],t_slide_df[t_value_index[i]]
                        t_X_t = t_slide_df_test[t_feature_index]
                        t_value = train_and_predict(t_X,t_y,t_X_t,t_model)
                        for t in range(len(t_value)): t_value[t] = int(t_value[t])
                        index[i].append(t_value)
                        print 'STATION: ',each
                        print 'AIR AQ TYPE:',names[i]
                        print 'VALUE:',t_value
                        print '------------------------------------'  
    if is_test == False:            
        # save to map
        position2values = {}
        for i in range(3):
            for j in range(len(file_list)):
                for k in range(48):
                    if len(file_list[j]) > 10: 
                        position_name = file_list[j][:10]
                    else: 
                        position_name = file_list[j]
                    if len(file_list) == 35:
                        t_key = str(names[i]) + '-' + position_name + '_aq#' + str(k)
                    else:
                        t_key = str(names[i]) + '-' + position_name + '#' + str(k)
                    position2values[t_key] = index[i][j][k]
    return  position2values

# 模型训练与预测

In [ ]:
def readwrite2(input_file,output_file):
    date_1=pd.read_csv(input_file,header=0,sep=',')
    date_1[['test_id','PM2.5', 'PM10', 'O3']].to_csv(output_file, sep=',', header=True,index=False)

def to_submit(input_path):
    files={'files': open(input_path,'rb')}

    data = {
        "user_id": "LiYouru",   #user_id is your username which can be found on the top-right corner on our website when you logged in.
        "team_token": "e4cbc74100e5eba99a9a3857d0424cbe38da5f51ca1274c7f604093f13f0a6d6", #your team_token.
        "description": 'big(72)',  #no more than 40 chars.
        "filename": "0524->0526-27", #your filename
    }
    
    url = 'https://biendata.com/competition/kdd_2018_submit/'
    response = requests.post(url, files=files, data=data)
    print response.text
    
def get_submition(n_model,time,n_pred):
    bj_value_map = cal_pred_each_pos(bj_index,n_model,False,n_pred)
    ld_value_map = cal_pred_each_pos(ld_index,n_model,False,n_pred)
    path = '/home/liyouru/.ipython/profile_myserver/dataset/KDD_Cup2018/testing_submissioin.csv'
    id_index = list(pd.read_csv(path)['test_id'])
    pm25,pm10,o3 = [],[],[]
    aq_index = [pm25,pm10,o3]
    names = ['PM2.5', 'PM10', 'O3']
    # get prediction from map
    for i in range(len(id_index)):
        for j in range(3):
            t_key = str(names[j]) + '-' + id_index[i]
            if i < 1680:
                aq_index[j].append(bj_value_map[t_key])
            else:
                aq_index[j].append(ld_value_map[t_key])
    
    submit_value = [id_index,pm25,pm10,o3]
    # negtitive value detection
    for i in range(len(id_index)):
        if pm25[i] < 0: pm25[i] = 0
        if pm10[i] < 0: pm10[i] = 0
        if o3[i] < 0: o3[i] = 0
    # save to csv     
    submit_names = ['test_id','PM2.5', 'PM10', 'O3']
    submit_df = DataFrame(np.transpose(submit_value) ,index = None, columns = submit_names)
    
    input_path = base_path + 'submittion/submissioin_05' + time + '01.csv'
    output_path = base_path + 'submittion/submissioin_05_' + time + '_1.csv'
    submit_df.to_csv(input_path)
    readwrite2(input_path, output_path)
#     to_submit(output_path)

In [ ]:
get_submition('GBRT','31',72)

100%|██████████| 11317/11317 [01:19<00:00, 143.07it/s]
/home/liyouru/virtual_tf1.3_python2.7/lib/python2.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/liyouru/virtual_tf1.3_python2.7/lib/python2.7/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/liyouru/virtual_tf1.3_python2.7/lib/python2.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/liyouru/virtual_tf1.3

TRAINING SIZE: (11198, 525) , 11198
GBRT Model is Training...
STATION:  dongsi
AIR AQ TYPE: PM2.5
VALUE: [30, 31, 40, 41, 44, 50, 53, 45, 54, 49, 47, 51, 52, 53, 54, 58, 57, 54, 49, 38, 37, 36, 38, 33, 30, 43, 30, 30, 33, 42, 43, 47, 48, 46, 44, 45, 55, 42, 46, 57, 69, 65, 60, 56, 52, 53, 46, 55]
------------------------------------
TRAINING SIZE: (11198, 525) , 11198
GBRT Model is Training...
STATION:  dongsi
AIR AQ TYPE: PM10
VALUE: [62, 61, 69, 65, 68, 64, 55, 67, 75, 84, 83, 73, 75, 75, 76, 98, 97, 91, 91, 87, 87, 82, 96, 92, 86, 86, 83, 91, 92, 95, 99, 96, 71, 75, 76, 87, 85, 82, 89, 97, 103, 108, 114, 125, 127, 123, 129, 117]
------------------------------------
TRAINING SIZE: (11198, 525) , 11198
GBRT Model is Training...
STATION:  dongsi
AIR AQ TYPE: O3
VALUE: [79, 85, 79, 91, 90, 91, 104, 98, 126, 131, 135, 122, 114, 111, 98, 82, 68, 68, 49, 49, 46, 55, 55, 47, 44, 51, 65, 110, 129, 134, 151, 170, 178, 177, 179, 190, 190, 185, 182, 176, 169, 147, 123, 105, 85, 71, 66, 72]
----

100%|██████████| 11317/11317 [01:18<00:00, 144.67it/s]


TRAINING SIZE: (11198, 525) , 11198
GBRT Model is Training...
STATION:  tiantan
AIR AQ TYPE: PM2.5
VALUE: [20, 17, 19, 24, 25, 27, 22, 26, 26, 28, 28, 30, 35, 40, 33, 37, 37, 33, 31, 24, 19, 25, 24, 24, 23, 29, 27, 32, 33, 42, 39, 41, 38, 39, 38, 38, 35, 33, 31, 31, 52, 44, 50, 47, 47, 49, 41, 52]
------------------------------------
TRAINING SIZE: (11198, 525) , 11198
GBRT Model is Training...
STATION:  tiantan
AIR AQ TYPE: PM10
VALUE: [61, 53, 61, 57, 70, 76, 74, 85, 88, 86, 86, 82, 77, 78, 80, 107, 106, 89, 97, 94, 96, 91, 112, 99, 98, 100, 99, 103, 99, 110, 101, 107, 113, 106, 98, 96, 95, 102, 97, 99, 100, 110, 112, 113, 109, 109, 103, 93]
------------------------------------
TRAINING SIZE: (11198, 525) , 11198
GBRT Model is Training...
STATION:  tiantan
AIR AQ TYPE: O3
VALUE: [85, 98, 95, 94, 91, 102, 110, 132, 156, 141, 154, 122, 101, 107, 87, 84, 73, 57, 48, 37, 31, 30, 39, 61, 24, 30, 61, 120, 129, 137, 156, 171, 173, 172, 173, 178, 179, 170, 181, 171, 150, 110, 82, 75, 50, 49,

100%|██████████| 11317/11317 [01:13<00:00, 153.21it/s]


...

TRAINING SIZE: (12075, 339) , 12075
GBRT Model is Training...
STATION:  ST5
AIR AQ TYPE: PM2.5
VALUE: [28, 27, 28, 27, 28, 26, 25, 26, 27, 24, 24, 24, 22, 22, 21, 20, 23, 24, 24, 25, 24, 30, 33, 32, 28, 27, 27, 24, 23, 23, 22, 21, 18, 18, 18, 18, 15, 11, 11, 12, 11, 10, 10, 11, 13, 12, 12, 11]
------------------------------------
TRAINING SIZE: (12075, 339) , 12075
GBRT Model is Training...
STATION:  ST5
AIR AQ TYPE: PM10
VALUE: [40, 40, 39, 38, 39, 39, 37, 38, 38, 38, 39, 36, 36, 37, 40, 38, 36, 35, 37, 36, 37, 37, 37, 34, 37, 33, 30, 29, 27, 22, 22, 22, 21, 20, 22, 19, 20, 19, 21, 21, 21, 20, 19, 20, 20, 20, 19, 19]
------------------------------------


100%|██████████| 12195/12195 [00:52<00:00, 232.86it/s]


TRAINING SIZE: (12076, 339) , 12076
GBRT Model is Training...
STATION:  TH4
AIR AQ TYPE: PM2.5
VALUE: [23, 24, 24, 24, 24, 23, 23, 23, 24, 22, 21, 21, 22, 24, 23, 24, 24, 23, 23, 21, 23, 25, 25, 24, 24, 24, 24, 23, 22, 18, 18, 17, 18, 14, 15, 15, 16, 12, 14, 11, 11, 10, 8, 10, 10, 10, 10, 10]
------------------------------------
TRAINING SIZE: (12076, 339) , 12076
GBRT Model is Training...
STATION:  TH4
AIR AQ TYPE: PM10
VALUE: [36, 38, 38, 38, 36, 36, 36, 38, 36, 36, 35, 38, 38, 37, 35, 33, 36, 38, 39, 37, 36, 39, 38, 34, 35, 37, 37, 39, 36, 31, 32, 31, 29, 27, 27, 26, 23, 20, 20, 19, 19, 19, 20, 20, 19, 18, 19, 19]
------------------------------------


100%|██████████| 12197/12197 [00:51<00:00, 236.60it/s]


TRAINING SIZE: (12078, 339) , 12078
GBRT Model is Training...
STATION:  MY7
AIR AQ TYPE: PM2.5
VALUE: [25, 25, 25, 25, 24, 23, 24, 23, 22, 23, 25, 25, 25, 25, 25, 25, 24, 23, 25, 24, 24, 24, 26, 26, 24, 21, 22, 21, 18, 17, 16, 16, 16, 17, 18, 19, 17, 15, 15, 14, 14, 12, 13, 14, 12, 12, 13, 13]
------------------------------------
TRAINING SIZE: (12078, 339) , 12078
GBRT Model is Training...
STATION:  MY7
AIR AQ TYPE: PM10
VALUE: [37, 38, 37, 38, 37, 37, 36, 38, 38, 36, 36, 36, 37, 35, 37, 34, 36, 36, 37, 35, 34, 34, 33, 36, 35, 34, 33, 33, 31, 32, 27, 28, 28, 29, 28, 29, 27, 23, 22, 20, 20, 20, 19, 21, 21, 22, 19, 19]
------------------------------------


# 结果提交

In [42]:
import requests
path = '/home/liyouru/.ipython/profile_myserver/dataset/KDD_Cup2018/submittion/submissioin_05_26_2.csv'
files={'files': open(path,'rb')}

data = {
    # liyouru
    "user_id": "LiYouru",   #user_id is your username which can be found on the top-right corner on our website when you logged in.
    "team_token": "e4cbc74100e5eba99a9a3857d0424cbe38da5f51ca1274c7f604093f13f0a6d6", #your team_token.
    "description": 'small(72)',  #no more than 40 chars.
    "filename": " 20180525->20180527-28", #your filename
}

url = 'https://biendata.com/competition/kdd_2018_submit/'

response = requests.post(url, files=files, data=data)

print(response.text)

{"success": true}
